In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# 读取消费记录和学生信息关联表
consumption = pd.read_csv('./data/task1_2_1.csv',encoding='gbk')

# 读取门禁记录和学生信息关联表
access = pd.read_csv('data/task1_2_2.csv',encoding='gbk')

# 按学生ID分组，提取每个学生的消费类别列表
consumption_grouped = consumption.groupby('CardNo')['Type'].apply(list).reset_index()

# 按学生ID分组，提取每个学生的门禁地点列表
access_grouped = access.groupby('CardNo')['Address'].apply(list).reset_index()

# 合并消费和门禁数据
merged = pd.merge(consumption_grouped, access_grouped, on='CardNo', how='inner')

# 将消费类别和门禁地点合并成一个列表
merged['items'] = merged.apply(lambda row: row['Type'] + row['Address'], axis=1)

# 将事务数据转换为二进制矩阵
te = TransactionEncoder()
te_ary = te.fit(merged['items']).transform(merged['items'])
df = pd.DataFrame(te_ary, columns=te.columns_)

# 生成频繁项集
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)

# 生成关联规则
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1,num_itemsets=2)

# 查看结果
print(rules)

      antecedents             consequents  antecedent support  \
0            (存款)                    (消费)            0.964571   
1            (消费)                    (存款)            1.000000   
2            (存款)             (第七教学楼[进门])            0.964571   
3     (第七教学楼[进门])                    (存款)            0.074286   
4            (存款)               (飞凤轩[出门])            0.964571   
..            ...                     ...                 ...   
85  (消费, 飞凤轩[出门])           (存款, 飞凤轩[进门])            0.581714   
86           (存款)  (消费, 飞凤轩[进门], 飞凤轩[出门])            0.964571   
87      (飞凤轩[进门])       (存款, 消费, 飞凤轩[出门])            0.585143   
88           (消费)  (存款, 飞凤轩[进门], 飞凤轩[出门])            1.000000   
89      (飞凤轩[出门])       (存款, 飞凤轩[进门], 消费)            0.581714   

    consequent support   support  confidence      lift  representativity  \
0             1.000000  0.964571    1.000000  1.000000               1.0   
1             0.964571  0.964571    0.964571  1.000000             

/root/miniconda3/lib/python3.12/site-packages/mlxtend/frequent_patterns/association_rules.py:182: RuntimeWarning: invalid value encountered in divide
  cert_metric = np.where(certainty_denom == 0, 0, certainty_num / certainty_denom)


第一行：

antecedents: （消费）

consequents: （存款）

antecedentsupport: 1.000000

consequentsupport: 0.964571

support: 0.964571

confidence: 0.964571

lift: 1.000000

leverage: 0.000000

conviction: 1.000000

这表示如果一个学生有“消费”，那么他有“存款”的规则。支持度是0.964571，置信度也是0.964571，提升度是1.0，杠杆是0，信度是1。

看起来这个规则的支持度很高，但提升度是1，表示没有关联，因为提升度等于1意味着前项和后项是独立的。可是支持度和置信度都很高，可能数据中有大部分学生都有存款，所以这种规则没有实际意义。

第二行：

antecedents: （存款）

consequents: （消费）

antecedentsupport: 0.964571

consequentsupport: 1.000000

support: 0.964571

confidence: 1.000000

lift: 1.000000

leverage: 0.000000

conviction: inf

这表示如果一个学生有“存款”，那么他有“消费”。支持度0.964571，置信度1.0，提升度1.0，杠杆0，信度是无穷大。

同样，大部分有存款的学生都有消费，但因为提升度是1，说明独立。

名词回忆：
  - 支持度(Support):表示规则在数据集中出现的频率，值越高表示规则越普通
  - 置信度(Confidence):表示前项出现的情况下，后项出现的概率，值越高表示规则越可靠
  - 提升度(Lift):表示规则的关联强度，值大于1表示正向关联，小于1表示负向关联，等于1表示无关联
  - 杠杆(Leverage):表示规则的有趣程度，值越高表示规则越有趣
  - 信度(Conviction):表示如果前项不出现，后项不出现的程度，值越高规则越强。

In [2]:
rules.to_csv('data/task4.csv',index=False,encoding='gbk')